In [3]:
import os
from nipype import Workflow, Node, Function, MapNode, JoinNode, config
config.enable_provenance()

In [4]:
def sum(a,b):
    return a + b

def concat(a, b):
    return [a, b]

def plus_one(a):
    return a + 1

def merge_and_scale_data(data2):
    import numpy as np
    return (np.array(data2) * 1000).tolist()

In [5]:
wf = Workflow('hello-world')

In [6]:
adder = Node(Function(input_names=['a', 'b'],
                     output_names=['sum'],
                     function=sum), name='a_plus_b')

'''
concater = Node(Function(input_names=['a', 'b'],
                     output_names=['some_list'],
                     function=concat), name='concat_a_b')

plusone = MapNode(Function(input_names=['a'],
                     output_names=['out'],
                     function=plus_one), 
                  iterfield=['a'],
                  name='add_1')
'''


joiner = JoinNode(Function(input_names=['data2'],
                          output_names=['data_scaled'],
                          function=merge_and_scale_data),
                 name='join_scale_data',
                 joinsource=adder,
                 joinfield=['data2'])

In [7]:
adder.iterables = ('a', [1, 2])
adder.inputs.b = 2

'''
wf.connect(adder, 'sum', concater, 'a')
concater.inputs.b = 3

wf.connect(concater, 'some_list', plusone, 'a')
'''

wf.connect(adder, 'sum', joiner, 'data2')

In [8]:
# wf.add_nodes([adder])

In [9]:
wf.base_dir = os.getcwd()
eg = wf.run(plugin='MultiProc', plugin_args={'n_procs': 2})

In [10]:
eg.nodes()

[hello-world.a_plus_b.aI.a0,
 hello-world.a_plus_b.aI.a1,
 hello-world.join_scale_data]

In [11]:
!ls

01_download.ipynb	 hello-world
02_inferfaces.ipynb	 hello-world.ipynb
03_nodes.ipynb		 index.ipynb
04_workflows.ipynb	 workflow_provenance_20160724T023217.provn
05_load_save_data.ipynb  workflow_provenance_20160724T023248.provn
06_iterables.ipynb	 workflow_provenance_20160724T023448.provn
data			 workflow_provenance_20160724T023448.trig


In [43]:
!nipype_display_crash crash-20160724-011821-main-add_1-3c81a7f7-d8f3-415f-bbf0-8a66e236de15.pklz



File: crash-20160724-011821-main-add_1-3c81a7f7-d8f3-415f-bbf0-8a66e236de15.pklz
Node: hello-world.add_1
Working directory: /home/main/notebooks/notebooks/hello-world/add_1


Node inputs:

a = [3, 3]
function_str = def plus_one(a):
    return a + 1

ignore_exception = False



Traceback: 
Traceback (most recent call last):
  File "/home/main/anaconda2/lib/python2.7/site-packages/nipype/pipeline/plugins/linear.py", line 39, in run
    node.run(updatehash=updatehash)
  File "/home/main/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 394, in run
    self._run_interface()
  File "/home/main/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 504, in _run_interface
    self._result = self._run_command(execute)
  File "/home/main/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 630, in _run_command
    result = self._interface.run()
  File "/home/main/anaconda2/lib/python2.7/site-packages/nipype/interfaces/ba

In [13]:
import rdflib as rl
g = rl.ConjunctiveGraph()
g.parse('workflow_provenance_20160724T023448.trig', format='trig')

<Graph identifier=file:///home/main/notebooks/notebooks/workflow_provenance_20160724T023448.trig (<class 'rdflib.graph.Graph'>)>

In [14]:
query = """
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX niiri: <http://iri.nidash.org/>
PREFIX nipype: <http://nipy.org/nipype/terms/>

# Find all activities that were involved in producing a file
SELECT ?alabel ?a_id ?blabel ?b_id
WHERE {

    ?a prov:qualifiedStart [ prov:starter ?b] .
    ?a_id a prov:Bundle;
        prov:wasGeneratedBy ?a .
    ?b_id a prov:Bundle;
        prov:wasGeneratedBy ?b .
    ?a rdfs:label ?alabel .
    ?b rdfs:label ?blabel .
}
"""

In [15]:
res = g.query(query)
print(res.serialize(format='csv'))

alabel,a_id,blabel,b_id
Function_join_scale_data,http://iri.nidash.org/30d0a36c514711e6ac830a580af40a0b,Function_a_plus_b,http://iri.nidash.org/30cda05e514711e6ac830a580af40a0b
Function_join_scale_data,http://iri.nidash.org/30d0a36c514711e6ac830a580af40a0b,Function_a_plus_b,http://iri.nidash.org/30cf1be6514711e6ac830a580af40a0b



In [16]:
graph_list = {}
for item in res.bindings:
    b_id = str(item['b_id'])
    a_id = str(item['a_id'])
    if a_id not in graph_list:
        graph_list[a_id] = []
    if b_id not in graph_list:
        graph_list[b_id] = []
    if b_id not in graph_list[a_id]:
        graph_list[a_id].append(b_id)
graph_list

{'http://iri.nidash.org/30cda05e514711e6ac830a580af40a0b': [],
 'http://iri.nidash.org/30cf1be6514711e6ac830a580af40a0b': [],
 'http://iri.nidash.org/30d0a36c514711e6ac830a580af40a0b': ['http://iri.nidash.org/30cda05e514711e6ac830a580af40a0b',
  'http://iri.nidash.org/30cf1be6514711e6ac830a580af40a0b']}

In [17]:
input_query = """
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX niiri: <http://iri.nidash.org/>
PREFIX nipype: <http://nipy.org/nipype/terms/>

# Find all activities that were involved in producing a file
SELECT ?in_port ?val ?in
WHERE {
    GRAPH <%s>
    {
    
    ?ic a nipype:Inputs;
        prov:hadMember ?in .
    ?in prov:value ?val .
    ?act prov:qualifiedUsage ?qual .
    ?qual nipype:inPort ?in_port; 
          prov:entity ?in .
    } .
}
"""

In [18]:
res = g.query(input_query % graph_list.keys()[0])
print(res.serialize(format='csv'))

in_port,val,in
ignore_exception,0,http://iri.nidash.org/9ec227a6f740d9ba54614c67f6f65e89
b,2,http://iri.nidash.org/f2674a4bd5d68a3c564b887c54369ff2
a,2,http://iri.nidash.org/f2674a4bd5d68a3c564b887c54369ff2
function_str,"def sum(a,b):
    return a + b
",http://iri.nidash.org/6e9ca8e35b79884a37bcefe4402e222a



In [19]:
output_query = """
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX niiri: <http://iri.nidash.org/>
PREFIX nipype: <http://nipy.org/nipype/terms/>

# Find all activities that were involved in producing a file
SELECT ?out_port ?val ?out
WHERE {
    GRAPH <%s>
    {
    
    ?oc a nipype:Outputs;
        prov:hadMember ?out;
        prov:wasGeneratedBy ?act .
    ?out prov:value ?val .
    ?out prov:qualifiedGeneration ?qual .
    ?qual nipype:outPort ?out_port; 
          prov:activity ?act .
    } .
}
"""

In [20]:
res = g.query(output_query % graph_list.keys()[0])
print(res.serialize(format='csv'))

out_port,val,out
sum,4,http://iri.nidash.org/5a554bd80b84b2a803d081008e96be30



In [21]:
for graph in graph_list:
    print("Node id: %s" % graph)
    print('Inputs')
    res = g.query(input_query % graph)
    print(res.serialize(format='csv'))
    print('Outputs')
    res = g.query(output_query % graph_list.keys()[0])
    print(res.serialize(format='csv'))

Node id: http://iri.nidash.org/30cf1be6514711e6ac830a580af40a0b
Inputs
in_port,val,in
ignore_exception,0,http://iri.nidash.org/9ec227a6f740d9ba54614c67f6f65e89
b,2,http://iri.nidash.org/f2674a4bd5d68a3c564b887c54369ff2
a,2,http://iri.nidash.org/f2674a4bd5d68a3c564b887c54369ff2
function_str,"def sum(a,b):
    return a + b
",http://iri.nidash.org/6e9ca8e35b79884a37bcefe4402e222a

Outputs
out_port,val,out
sum,4,http://iri.nidash.org/5a554bd80b84b2a803d081008e96be30

Node id: http://iri.nidash.org/30cda05e514711e6ac830a580af40a0b
Inputs
in_port,val,in
function_str,"def sum(a,b):
    return a + b
",http://iri.nidash.org/6e9ca8e35b79884a37bcefe4402e222a
b,2,http://iri.nidash.org/f2674a4bd5d68a3c564b887c54369ff2
ignore_exception,0,http://iri.nidash.org/9ec227a6f740d9ba54614c67f6f65e89
a,1,http://iri.nidash.org/e5692843450c8e664dfe4b40a35647de

Outputs
out_port,val,out
sum,4,http://iri.nidash.org/5a554bd80b84b2a803d081008e96be30

Node id: http://iri.nidash.org/30d0a36c514711e6ac830a580af40a0b
